<a href="https://colab.research.google.com/github/qwertymasterkey/captcha_recognition/blob/main/best_resnet50Captcha_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pathlib import Path
data_dir = Path("/content/drive/MyDrive/captcha_segment/segmented_dataset_")
images = sorted(list(map(str, list(data_dir.glob("*.jpg")))))

In [ ]:
!pip install split_folders
import splitfolders


In [ ]:
input_folder="/content/drive/MyDrive/captcha_segment/segmented_dataset_/captcha/data/"
output_folder="/content/drive/MyDrive/captcha_segment/splitted_dataset_/"


In [ ]:
splitfolders.ratio(input_folder,output_folder,seed=5,ratio=(.7,.15,.15))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
img_height,img_width=(224,224)
batch_size=32
train_data_dir="/content/drive/MyDrive/captcha_segment/splitted_dataset_/train"
val_data_dir="/content/drive/MyDrive/captcha_segment/splitted_dataset_/val"
test_data_dir="/content/drive/MyDrive/captcha_segment/splitted_dataset_/test"

In [ ]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 rotation_range=20,
                                 horizontal_flip=True,)

In [ ]:
train_generator=train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 4072 images belonging to 62 classes.


In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
valid_generator=test_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical',
)

Found 849 images belonging to 62 classes.


In [ ]:
test_generator=test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height,img_width),
    batch_size=1,
    class_mode='categorical',
)

Found 937 images belonging to 62 classes.


In [ ]:
x,y=test_generator.next()

In [ ]:
x.shape

(1, 224, 224, 3)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization,GlobalAveragePooling2D
import numpy as np
from tensorflow.keras import layers


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation,BatchNormalization
import os
import pickle

In [ ]:
train_generator.num_classes

62

In [ ]:
base_model=ResNet50(include_top=False,weights='imagenet')
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
x=Dropout(0.2)(x)
predictions=Dense(train_generator.num_classes,activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=predictions)

for layer in base_model.layers:
  layer.trainable=True
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

94781440/94765736 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                             

In [ ]:
filepath = 'my_best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

In [ ]:
callbacks = [checkpoint]
model.fit(train_generator,
          epochs = 50,
          validation_data = valid_generator,
          callbacks=callbacks
          )

Epoch 1/50
128/128 [==============================] - ETA: 0s - loss: 2.1104 - accuracy: 0.4661
Epoch 1: val_loss improved from inf to 23.77749, saving model to my_best_model.epoch01-loss23.78.hdf5
128/128 [==============================] - 932s 7s/step - loss: 2.1104 - accuracy: 0.4661 - val_loss: 23.7775 - val_accuracy: 0.0813
Epoch 2/50
128/128 [==============================] - ETA: 0s - loss: 0.7057 - accuracy: 0.7755
Epoch 2: val_loss improved from 23.77749 to 1.58624, saving model to my_best_model.epoch02-loss1.59.hdf5
128/128 [==============================] - 70s 544ms/step - loss: 0.7057 - accuracy: 0.7755 - val_loss: 1.5862 - val_accuracy: 0.5630
Epoch 3/50
128/128 [==============================] - ETA: 0s - loss: 0.4257 - accuracy: 0.8671
Epoch 3: val_loss improved from 1.58624 to 0.59707, saving model to my_best_model.epoch03-loss0.60.hdf5
128/128 [==============================] - 70s 542ms/step - loss: 0.4257 - accuracy: 0.8671 - val_loss: 0.5971 - val_accuracy: 0.8339


In [ ]:
test_loss,test_acc=model.evaluate(test_generator,verbose=1)

937/937 [==============================] - 190s 203ms/step - loss: 1.0601 - accuracy: 0.8079


In [ ]:
from tensorflow.keras.models import load_model
best_model=tf.keras.models.load_model('/content/my_best_model.epoch46-loss0.08.hdf5')

OSError: ignored

In [ ]:
test_loss,test_acc=best_model.evaluate(test_generator,verbose=1)